In [14]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import yfinance as yf
import copy
import random
from pymongo import MongoClient
import sys
from dotenv import load_dotenv
import warnings
import time

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

load_dotenv()
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
stock_db = mongo_client["stock_db"]
ref_db = mongo_client["ref_db"]

#### Chuẩn bị các dữ liệu

##### Các dữ liệu dùng để làm map tham chiếu

In [15]:
def get_mongo_df(db_collection, df_name, find_query=None, projection=None):
    # Truy cập collection
    collection = db_collection[df_name]
    # Nếu không truyền vào find_query thì mặc định lấy tất cả document
    if find_query is None:
        find_query = {}
    # Nếu không truyền vào projection thì mặc định loại bỏ trường _id
    if projection is None:
        projection = {"_id": 0}
    # Thực hiện lệnh find với điều kiện và projection đã cho
    docs = collection.find(find_query, projection)
    # Chuyển đổi kết quả sang DataFrame và trả về
    df = pd.DataFrame(list(docs))
    return df

In [16]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = get_mongo_df(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = get_mongo_df(ref_db, "order_map")
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = get_mongo_df(ref_db, "group_map")
group_map_dict = group_map.set_index('code')['group'].to_dict()

#Tạo các danh sách nhóm trong mỗi cách chia cổ phiếu
all_stock_key_list = [key for key, value in group_map_dict.items() if value == 'all']
industry_name_list = [key for key, value in group_map_dict.items() if value in ['hsA', 'hsB', 'hsC', 'hsD']]
industry_perform_list = [key for key, value in group_map_dict.items() if value == 'hs']
marketcap_group_list = [key for key, value in group_map_dict.items() if value == 'cap']

#Tạo danh danh key cho tổng tất cả các nhóm
group_stock_key_list = all_stock_key_list + industry_name_list + industry_perform_list + marketcap_group_list

In [17]:
def get_file_name_list(folder_path):
    file_name_list = []
    files = os.listdir(folder_path)
    for file in files:
        file_name_list.append(file[:-4])
    return file_name_list

def filter_market_file_name_list(file_name_list):
    filtered_list = [item for item in file_name_list if not (item.endswith('_AC') or item.endswith('_CC'))]
    return filtered_list

eod_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\stock"
eod_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\index"
eod_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\futures"
itd_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\stock"
itd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\index"
itd_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\futures"
nn_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\foreign"
td_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\prop"
nntd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\market"
other_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\other"

In [18]:
#Tạo dict map thời gian và số lượng cổ phiếu
quarter_map = get_mongo_df(ref_db, 'quarter_map')
quarter_map_dict = quarter_map.set_index('quarter').apply(lambda row: row.tolist(), axis=1).to_dict()

#Xoá đi quý hiện tại để chỉ tính toán tới quý trước đó
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
        previous_year = year - 1
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q1"
        previous_year = year
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
        previous_year = year
    else:
        quarter = "q4"
        previous_quarter = "q3"
        previous_year = year
    
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        return f'{previous_quarter}_{previous_year}'

#Lấy ra list cổ phiếu của giai đoạn hiện tại
quarter_stock_map = get_mongo_df(ref_db, 'quarter_stock_map')
total_stock_list = get_mongo_df(ref_db, 'current_total_stock_list')['ticker'].tolist()
current_quarter_stock_list = list(set(get_file_name_list(itd_stock_folder_path)) 
                                & set(quarter_stock_map[get_quarter('current_quarter')].dropna().tolist()))

#Lấy ra khoảng thời gian tính toán cho quý này và quý trước
calculate_time_span = [quarter_map_dict['q2_2020'][0], quarter_map_dict[get_quarter('current_quarter')][1]]
current_quarter_span = [quarter_map_dict[get_quarter('current_quarter')][0], quarter_map_dict[get_quarter('current_quarter')][1]]
previous_quarter_span = [quarter_map_dict[get_quarter('previous_quarter')][0], quarter_map_dict[get_quarter('previous_quarter')][1]]

##### Các biến thời gian

In [19]:
#Tạo date_series cho thời gian tính toán
date_series = get_mongo_df(ref_db, 'date_series')

#Xác định ngày hiện tại
today = date_series['date'].iloc[0]

##### Đọc dữ liệu nhóm từ MongoDB

In [20]:
#Đọc dữ liệu điểm dòng tiền âm dương nhóm cổ phiếu
eod_signed_group_df = get_mongo_df(stock_db, "eod_signed_group")
history_signed_group_df = get_mongo_df(stock_db, "history_signed_group")
itd_signed_group_df = get_mongo_df(stock_db, "itd_signed_group")
full_signed_group_df = pd.concat([eod_signed_group_df, history_signed_group_df], axis=0).reset_index(drop=True)

#Đọc dữ liệu ms lịch sử
eod_ms_chart_df = get_mongo_df(stock_db, "eod_ms_chart")
history_ms_chart_df = get_mongo_df(stock_db, "history_ms_chart")
full_ms_chart_df = pd.concat([eod_ms_chart_df, history_ms_chart_df], axis=0).reset_index(drop=True)

#### Tín hiệu toàn thị trường

##### Tạo bảng market phase

- Các hàm tính toán

In [21]:
def calculate_market_ms(ms_df):
    # Tạo bản sao của DataFrame để không làm thay đổi dữ liệu gốc
    df = ms_df.copy()

    # --- Tính toán các cột shift cho trend_5p ---
    df['5p_shift1'] = df['trend_5p'].shift(-1)
    df['5p_shift2'] = df['trend_5p'].shift(-2)
    df['5p_shift5'] = df['trend_5p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1)
    df['5p_diff1'] = df['trend_5p'][::-1].diff()[::-1]
    df['5p_diff5'] = df['trend_5p'] - df['5p_shift5']
    
    # --- Tính toán các cột shift cho trend_20p ---
    df['20p_shift1'] = df['trend_20p'].shift(-1)
    df['20p_shift2'] = df['trend_20p'].shift(-2)
    df['20p_shift5'] = df['trend_20p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1)
    df['20p_diff1'] = df['trend_20p'][::-1].diff()[::-1]
    df['20p_diff5'] = df['trend_20p'] - df['20p_shift5']

    # --- Tính toán các cột shift cho trend_60p ---
    df['60p_shift1'] = df['trend_60p'].shift(-1)
    df['60p_shift2'] = df['trend_60p'].shift(-2)

    # --- Tín hiệu mua (buy) ---
    # So sánh giá trị hiện tại với các giá trị dịch chuyển
    df['buy_ms5p_check'] = ((df['trend_5p'] > df['5p_shift1']) & (df['trend_5p'] > df['5p_shift5'])).astype(int)
    df['buy_ms20p_check'] = ((df['trend_20p'] > df['20p_shift1']) & (df['trend_20p'] > df['20p_shift5'])).astype(int)
    df['buy_ms60p_check'] = ((df['trend_60p'] > df['60p_shift1']) & (df['trend_60p'] > df['60p_shift2'])).astype(int)

    # Kiểm tra xem giá có ở mức quá cao không
    df['buy_overprice_check'] = ((df['5p_diff1'] > 0.4) | ((df['trend_5p'] < 0.9) & (df['trend_20p'] < 0.9))).astype(int)

    # Kiểm tra giá trị biến động của xu hướng
    buy_threshold_5p = 0.3
    buy_threshold_20p = 0.2
    df['buy_msvalue_check'] = ( (df['5p_diff1'] > 0.4) | 
            (((df['5p_shift1'] > buy_threshold_5p) & (df['trend_5p'] > buy_threshold_5p)) | ((df['5p_shift1'] < buy_threshold_5p) & (df['trend_5p'] > buy_threshold_5p) & (df['5p_diff1']*0.3 < (df['trend_5p'] - buy_threshold_5p)))) &
            (((df['20p_shift1'] > buy_threshold_20p) & (df['trend_20p'] > buy_threshold_20p)) | ((df['20p_shift1'] < buy_threshold_20p) & (df['trend_20p'] > buy_threshold_20p) & (df['20p_diff1']*0.3 < (df['trend_20p'] - buy_threshold_20p))))
        ).astype(int)

    # Tính tín hiệu mua dựa trên tất cả các điều kiện
    df['buy_check'] = df.apply(
        lambda x: 1 if (x['buy_ms5p_check'] == 1 and x['buy_ms20p_check'] == 1 and 
                        x['buy_ms60p_check'] == 1 and x['buy_overprice_check'] == 1 and 
                        x['buy_msvalue_check'] == 1)
                  else (2 if x['buy_overprice_check'] == 0 else 0),
        axis=1
    )

    # --- Tín hiệu bán (sell) ---
    df['sell_ms5p_check'] = ((df['trend_5p'] < df['5p_shift1']) | (df['trend_5p'] < df['5p_shift2'])).astype(int)
    df['sell_ms20p_check'] = ((df['trend_20p'] < df['20p_shift1']) | (df['trend_20p'] < df['20p_shift2'])).astype(int)

    # Kiểm tra xem giá có ở mức quá thấp không (underprice)
    df['sell_underprice_check'] = ((df['trend_5p'] > 0.22) | (df['trend_20p'] > 0.25)).astype(int)

    # Kiểm tra giá trị biến động cho tín hiệu bán
    sell_threshold_5p = 0.4
    sell_threshold_20p = 0.5
    df['sell_msvalue_check'] = ( (df['5p_diff1'] < -0.4) |
                                
        (((df['5p_shift1'] < sell_threshold_5p) & (df['trend_5p'] < sell_threshold_5p)) | ((df['5p_shift1'] > sell_threshold_5p) & (df['trend_5p'] < sell_threshold_5p) & (-df['5p_diff1']*0.3 < (sell_threshold_5p - df['trend_5p'])))) &
        (((df['20p_shift1'] < sell_threshold_20p) & (df['trend_20p'] < sell_threshold_20p)) | ((df['20p_shift1'] > sell_threshold_20p) & (df['trend_20p'] < sell_threshold_20p) & (-df['20p_diff1']*0.3 < (sell_threshold_20p - df['trend_20p'])))) &
        
        (df['trend_60p'] < 0.7)
    ).astype(int)
    
    # Tính tín hiệu bán dựa trên tất cả các điều kiện
    df['sell_check'] = df.apply(
        lambda x: 1 if (x['sell_ms5p_check'] == 1 and x['sell_ms20p_check'] == 1 and 
                        x['sell_underprice_check'] == 1 and x['sell_msvalue_check'] == 1)
                  else (2 if x['sell_underprice_check'] == 0 else 0),
        axis=1
    )

    return df[['date','trend_5p','trend_20p','trend_60p','buy_ms5p_check','buy_ms20p_check','buy_ms60p_check','buy_overprice_check','buy_msvalue_check','sell_ms5p_check','sell_ms20p_check','sell_underprice_check','sell_msvalue_check','buy_check','sell_check']]

In [22]:
def calculate_market_phase_raw(df):
    """
    Tính toán cột 'market_phase_raw' dựa trên các điều kiện:
      - Nếu 'buy_check' == 2         => market_phase_raw = 2
      - Nếu 'buy_check' == 1 và 'score_check' == 1 => market_phase_raw = 1
      - Nếu 'sell_check' == 1        => market_phase_raw = -1
      - Nếu 'sell_check' == 2        => market_phase_raw = -2
      - Các trường hợp khác           => market_phase_raw = NaN

    Sau đó, thực hiện backfill (bfill) cho các giá trị thiếu (NaN) và ép dòng cuối cùng về 1.
    
    Parameters:
      df (DataFrame): DataFrame chứa các cột 'buy_check', 'score_check', 'sell_check'.
    
    Returns:
      Series: cột 'market_phase_raw' sau khi tính toán và xử lý.
    """
    df_copy = df.copy()

    conditions = [
        df_copy['buy_check'] == 2,
        (df_copy['buy_check'] == 1) & (df_copy['score_check'] == 1),
        df_copy['sell_check'] == 1,
        df_copy['sell_check'] == 2
    ]
    choices = [2, 1, -1, -2]
    
    # Tạo cột market_phase_raw theo điều kiện, nếu không thỏa thì gán NaN
    df_copy['market_phase_raw'] = np.select(conditions, choices, default=np.nan)
    
    # Sử dụng backfill để lấp đầy các giá trị NaN
    df_copy['market_phase_raw'] = df_copy['market_phase_raw'].bfill()
    
    # Đảm bảo dòng cuối cùng luôn là 1 (mua)
    df_copy.loc[df_copy.index[-1], 'market_phase_raw'] = 1
    
    return df_copy['market_phase_raw']

In [ ]:
def adjust_market_phase(df):
    """
    Hàm nhận vào DataFrame có cột 'market_phase_raw' (các giá trị: 1, 2, -1, -2)
    và tạo cột binary (1: mua, 0: bán) theo quy tắc kiểm tra từ dưới lên trên với “khóa” trạng thái:
    
      - Dòng cuối (thời gian mới nhất) được xác định theo dấu của market_phase_raw:
            Nếu > 0 -> 1 (mua), nếu < 0 -> 0 (bán).
      - Khi duyệt ngược từ dưới lên trên:
          + Nếu trạng thái hiện hành là mua (state > 0):
                - Nếu gặp tín hiệu âm:
                      * Nếu tín hiệu là -1: chuyển sang bán (state = -1).
                      * Nếu tín hiệu là -2: giữ mua (không cập nhật state).
                - Nếu gặp tín hiệu dương:
                      * Nếu state hiện hành là 2 và tín hiệu mới là 1, thì giữ lại state = 2.
                      * Các trường hợp khác: cập nhật state = tín hiệu mới.
          + Nếu trạng thái hiện hành là bán (state < 0):
                - Nếu gặp tín hiệu dương:
                      * Nếu tín hiệu là 1: chuyển sang mua (state = 1).
                      * Nếu tín hiệu là 2: giữ bán (không cập nhật state).
                - Nếu gặp tín hiệu âm:
                      * Nếu state hiện hành là -2 và tín hiệu mới là -1, thì giữ lại state = -2.
                      * Các trường hợp khác: cập nhật state = tín hiệu mới.
          + Sau mỗi bước, gán binary[i] = 1 nếu state > 0, ngược lại 0.
    """
    phases = df['market_phase_raw'].tolist()
    n = len(phases)
    binary = [None] * n

    # Dòng cuối cùng: xác định trạng thái theo tín hiệu của nó
    state = phases[-1]
    binary[-1] = 1 if state > 0 else 0

    # Duyệt ngược từ dòng áp chót về đầu DataFrame
    for i in range(n - 2, -1, -1):
        p = phases[i]
        if state > 0:  # Hiện đang ở trạng thái mua
            if p < 0:
                # Nếu tín hiệu âm: chỉ chuyển nếu là -1 (đảo chiều)
                if p == -1:
                    state = -1
                elif p == -2:
                    # Tín hiệu -2 không làm chuyển trạng thái, giữ mua
                    pass
            elif p > 0:
                # Nếu tín hiệu dương: kiểm tra "khóa" trạng thái mua
                # Nếu đang ở phase 2 và tín hiệu mới là 1, thì giữ phase 2 (không giảm xuống phase 1)
                if state == 2 and p == 1:
                    pass
                else:
                    state = p
            # Nếu p == 0 (không có tín hiệu) thì giữ nguyên state
        else:  # state < 0, hiện đang ở trạng thái bán
            if p > 0:
                # Nếu tín hiệu dương: chỉ chuyển nếu tín hiệu là 1 (đảo chiều)
                if p == 1:
                    state = 1
                elif p == 2:
                    # Tín hiệu 2 không làm chuyển trạng thái, giữ bán
                    pass
            elif p < 0:
                # Nếu tín hiệu âm: kiểm tra "khóa" trạng thái bán
                # Nếu đang ở phase -2 và tín hiệu mới là -1, thì giữ phase -2 (không giảm xuống -1)
                if state == -2 and p == -1:
                    pass
                else:
                    state = p
            # Nếu p == 0 thì giữ nguyên
        binary[i] = 1 if state > 0 else 0

    return binary


In [ ]:
def final_market_phase_T3(df):
    # Tạo bản sao của DataFrame để không làm thay đổi dữ liệu gốc
    df_copy = df.copy()
    
    # Tạo cột market_phase_final dựa trên cột market_phase gốc
    df_copy['market_phase_final'] = df_copy['market_phase_adj']
    # Tạo cột tín hiệu T3, khởi tạo giá trị bằng 0
    df_copy['t3_signal'] = 0

    # Lặp ngược từ len(df_copy)-4 đến 0 để đảm bảo các phiên sau đã được xử lý
    for i in range(len(df_copy) - 4, -1, -1):
        current = df_copy['market_phase_final'].iloc[i]
        next_val = df_copy['market_phase_final'].iloc[i+1]
        next2 = df_copy['market_phase_final'].iloc[i+2]
        next3 = df_copy['market_phase_final'].iloc[i+3]

        # Nếu giá trị của phiên tiếp theo lớn hơn phiên sau và lớn hơn phiên hiện tại
        if (next_val > next2) and (next_val > current):
            # Cập nhật market_phase_final tại vị trí i với giá trị của phiên tiếp theo
            df_copy.at[df_copy.index[i], 'market_phase_final'] = next_val
            # Gán tín hiệu T3 bằng 2 (còn 2 phiên nữa mới bán được)
            df_copy.at[df_copy.index[i], 't3_signal'] = 2
        # Nếu giá trị của phiên thứ 2 sau lớn hơn phiên thứ 3 và phiên tiếp theo lớn hơn phiên hiện tại
        elif (next2 > next3) and (next_val > current):
            df_copy.at[df_copy.index[i], 'market_phase_final'] = next_val
            # Gán tín hiệu T3 bằng 1 (còn 1 phiên nữa mới bán được)
            df_copy.at[df_copy.index[i], 't3_signal'] = 1

    # Trả về 2 cột mới: market_phase_final và t3_signal
    return df_copy['market_phase_final']


- Tạo bảng market phase

In [ ]:
#Tách bảng điểm dòng tiền thanh bảng dòng tiền âm và dương
group_positive_df = full_signed_group_df[full_signed_group_df['type']=='pos'].drop(columns=['type']).reset_index(drop=True)
group_negative_df = full_signed_group_df[full_signed_group_df['type']=='neg'].drop(columns=['type']).reset_index(drop=True)

#Tính phân bổ vốn hoá tổng chưa điều chỉnh
market_phase_df = date_series.copy()

#Tính dòng tiền vào và ra trung trung trong 5 phiên
market_phase_df[['hsA+','hsB+','hsC+','hsD+']] = group_positive_df[industry_perform_list][::-1].rolling(window=5, min_periods=1).mean()[::-1]
market_phase_df[['hsA-','hsB-','hsC-','hsD-']] = group_negative_df[industry_perform_list][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

#Tính tỉ lệ dòng tiền vào/ra, nhân theo tỉ lệ và tạo cột score check để kiểm tra điều kiện dòng tiền
market_phase_df['hsA_score'] = ((market_phase_df['hsA+'] - market_phase_df['hsA-']) / (market_phase_df['hsA+'] + market_phase_df['hsA-'])) * 0.3
market_phase_df['hsB_score'] = (market_phase_df['hsB+'] - market_phase_df['hsB-']) / (market_phase_df['hsB+'] + market_phase_df['hsB-']) * 0.3
market_phase_df['hsC_score'] = (market_phase_df['hsC+'] - market_phase_df['hsC-']) / (market_phase_df['hsC+'] + market_phase_df['hsC-']) * 0.3
market_phase_df['hsD_score'] = (market_phase_df['hsD+'] - market_phase_df['hsD-']) / (market_phase_df['hsD+'] + market_phase_df['hsD-']) * 0.1

market_phase_df['hsC_score_S1'] = market_phase_df['hsC_score'].shift(-1).ffill()

market_phase_df['market_score'] = market_phase_df[['hsA_score','hsB_score','hsC_score','hsD_score']].sum(axis=1)
market_phase_df['score_check'] = market_phase_df.apply(lambda x: 1 if (x['market_score'] > 0.2) & (x['hsA_score'] > 0.12)  & (x['hsC_score'] > x['hsC_score_S1']) else 0, axis=1)


market_phase_df = market_phase_df.drop(columns=['hsA+', 'hsB+', 'hsC+', 'hsD+', 'hsA-', 'hsB-', 'hsC-', 'hsD-', 'hsC_score_S1'])

#Ghép bảng phase với bảng MS để thêm các cột tín hiệu từ MS
market_ms_chart_df = full_ms_chart_df[full_ms_chart_df['ticker']=='all'][['date','trend_5p','trend_20p','trend_60p']]
market_ms_chart_df = calculate_market_ms(market_ms_chart_df).sort_values('date', ascending=False).reset_index(drop=True)
market_phase_df = market_phase_df.merge(market_ms_chart_df, on='date', how='left')

#Điều chỉnh lại các giai đoạn mua bán tạo thành 2 giai đoạn chính mua (1) và bán (0)
market_phase_df['market_phase_raw'] = calculate_market_phase_raw(market_phase_df)
market_phase_df['market_phase_adj'] = adjust_market_phase(market_phase_df)
market_phase_df['market_phase_final'] = final_market_phase_T3(market_phase_df)

#### Lưu dữ liệu

In [ ]:
def overwrite_mongo(collection, df):
    # Lấy tên collection hiện tại và database
    collection_name = collection.name
    db = collection.database  # Truy cập database từ collection
    temp_collection_name = f"temp_{collection_name}"
    old_collection_name = f"old_{collection_name}"

    # Reset index của DataFrame
    df = df.reset_index(drop=True)
    records = df.replace({pd.NaT: None}).to_dict(orient='records')

    # 1. Lưu dữ liệu vào collection tạm
    temp_collection = db[temp_collection_name]
    temp_collection.drop()  # Đảm bảo collection tạm sạch trước khi insert
    temp_collection.insert_many(records)

    # 2. Rename collection cũ thành 'old_' (nếu tồn tại)
    if collection_name in db.list_collection_names():
        db[collection_name].rename(old_collection_name, dropTarget=True)

    # 3. Rename collection tạm thành tên chuẩn
    temp_collection.rename(collection_name, dropTarget=True)

    # 4. Xóa collection 'old_' (nếu tồn tại)
    if old_collection_name in db.list_collection_names():
        db[old_collection_name].drop()
        
overwrite_mongo(stock_db["market_phase"], market_phase_df)
time.sleep(120)